In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install biopython

     |████████████████████████████████| 2.3MB 12.5MB/s 


In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from Bio.SeqIO import parse
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from sklearn.preprocessing import LabelEncoder
import re
import keras
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from collections import Counter
from sklearn.utils import class_weight
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [ ]:
df = pd.read_csv('/content/drive/My Drive/WA_removed.csv')

In [ ]:
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))

LabelEncoder()

In [ ]:
def ordinal_encoder(my_array):
    
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, z
    
    return float_encoded

In [ ]:
data=df[["Sequence","Geo_Location"]]
data=data[data["Sequence"].notna()]

dummy=[]
dum=np.array(dummy)

form={"inp_seq":dum}

seq_df = pd.DataFrame (form, columns = ['inp_seq'])

seq_list=[]

for idx, seq in enumerate(list(data["Sequence"])):
    arr=ordinal_encoder(string_to_array(seq))
    seq_list.append(arr)
    
seq_df["inp_seq"]=seq_list

final_data= data.assign(enc_seq=seq_df)

final_data=final_data[["enc_seq","Geo_Location"]]

final_data

,enc_seq,Geo_Location
0,"[0.75, 0.75, 1.0, 1.0, 1.0, 0.25, 1.0, 0.25, 0...",USA
1,"[0.25, 1.0, 0.25, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5...",USA
2,"[0.25, 0.25, 0.75, 0.75, 1.0, 1.0, 1.0, 0.25, ...",USA
3,"[0.75, 0.75, 1.0, 0.25, 0.25, 0.5, 0.25, 0.25,...",USA
4,"[0.75, 0.5, 1.0, 1.0, 0.25, 0.5, 0.75, 0.75, 1...",USA
...,...,...
5592,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",California
5593,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",California
5594,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Others
5595,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",Others


In [ ]:
def get_maxLen(enc_seq):
    
    max=0
    for row in enc_seq:
        #print(type(row))
        if(len(row)>max):
            max=len(row)
    
    
    return max

In [ ]:
def append_arr(enc_seq, max_len):
    
    seq_l=list(enc_seq)
    for i in range(len(seq_l),max_len):
        seq_l.append(0)
        
    new_seq_ar=np.array(seq_l)

        
    return new_seq_ar

In [ ]:
import math

max_len=get_maxLen(final_data["enc_seq"])
print("max_len is",max_len)
if max_len%2 !=0:
    max_len += 1

n = 1
for i in range(1, math.floor(math.sqrt(max_len))):
    if max_len%i==0:
        n = i

padded_seq_list=[]

for index, row in final_data.iterrows():
    seq_ar=append_arr(row["enc_seq"],max_len)
    padded_seq_list.append(seq_ar)

dummy=[]
dum=np.array(dummy)
form={"padded_enc_seq":dum}
padded_seq_df = pd.DataFrame (form, columns = ['padded_enc_seq'])

padded_seq_df["padded_enc_seq"]=padded_seq_list

padded_final_data= final_data.assign(padded_enc_seq=padded_seq_df)

padded_final_data=padded_final_data[["padded_enc_seq","Geo_Location"]]

padded_final_data.head(5)

max_len is 29921


,padded_enc_seq,Geo_Location
0,"[0.75, 0.75, 1.0, 1.0, 1.0, 0.25, 1.0, 0.25, 0...",USA
1,"[0.25, 1.0, 0.25, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5...",USA
2,"[0.25, 0.25, 0.75, 0.75, 1.0, 1.0, 1.0, 0.25, ...",USA
3,"[0.75, 0.75, 1.0, 0.25, 0.25, 0.5, 0.25, 0.25,...",USA
4,"[0.75, 0.5, 1.0, 1.0, 0.25, 0.5, 0.75, 0.75, 1...",USA


In [ ]:
def reshape_seq(seq, m ,n):
    
    r_seq=np.reshape(seq,(m,n))
    
    return r_seq

re_seqList=[]

for index, row in padded_final_data.iterrows():
    seq_ar=reshape_seq(row["padded_enc_seq"],n,max_len//n)
    re_seqList.append(seq_ar)

In [ ]:
X = np.asarray(re_seqList)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2], 1))
X.shape

(5597, 6, 4987, 1)

In [ ]:
Y = padded_final_data['Geo_Location'].values
Y.shape

(5597,)

In [ ]:
loc_classes = list(Y)
loc_classes = np.array(loc_classes) 
loc_classes = list(np.unique(loc_classes))

In [ ]:
df = padded_final_data['Geo_Location'].apply(loc_classes.index)
Y = df.values
Y.shape

(5597,)

In [ ]:
!pip install keras-metrics

In [ ]:
import keras
import keras_metrics
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from collections import Counter
from sklearn.utils import class_weight
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.30)

In [ ]:
print("Train X Shape:",x_train.shape)
print("Train Y Shape:",y_train.shape)
print("Validation X Shape:",x_val.shape)
print("Validation Y Shape:",y_val.shape)

Train X Shape: (3917, 6, 4987, 1)
Train Y Shape: (3917,)
Validation X Shape: (1680, 6, 4987, 1)
Validation Y Shape: (1680,)


In [ ]:
batch_size = 64
num_classes = 10
epochs = 10

img_rows, img_cols = x_train.shape[1], x_train.shape[2]

x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
x_val = x_val.reshape(x_val.shape[0],img_rows,img_cols,1)
print(y_train.shape)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'validation samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

(3917,)
x_train shape: (3917, 6, 4987, 1)
3917 train samples
1680 validation samples


In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3,3), activation='relu', input_shape=(img_rows,img_cols,1)))
model.add(Conv2D(16, (3, 3),activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3,3), padding='same',activation='relu'))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

In [ ]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 4, 4985, 16)       160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 4983, 16)       2320      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 4983, 32)       4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 4983, 32)       9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 2491, 32)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 2491, 32)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 79712)            

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=200, verbose=1, validation_data=(x_val, y_val))
score = model.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 3917 samples, validate on 1680 samples
Epoch 1/200
3917/3917 [==============================] - 12s 3ms/step - loss: 2.1667 - accuracy: 0.2109 - val_loss: 2.0726 - val_accuracy: 0.3369
Epoch 2/200
3917/3917 [==============================] - 4s 1ms/step - loss: 2.0346 - accuracy: 0.3143 - val_loss: 1.8653 - val_accuracy: 0.3738
Epoch 3/200
3917/3917 [==============================] - 4s 1ms/step - loss: 1.9223 - accuracy: 0.3431 - val_loss: 1.7755 - val_accuracy: 0.3833
Epoch 4/200
3917/3917 [==============================] - 4s 1ms/step - loss: 1.8355 - accuracy: 0.3975 - val_loss: 1.7423 - val_accuracy: 0.4363
Epoch 5/200
3917/3917 [==============================] - 4s 1ms/step - loss: 1.7707 - accuracy: 0.4080 - val_loss: 1.7522 - val_accuracy: 0.4506
Epoch 6/200
3917/3917 [==============================] - 4s 1ms/step - loss: 1.7206 - accuracy: 0.4266 - val_loss: 1.6025 - val_accuracy: 0.4565
Epoch 7/200
3917/3917 [==============================] - 4s 1ms/step - loss: 1.68

In [ ]:
Y_pred = model.predict(x_val)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion matrix - Test')
y_val2 = np.argmax(y_val,axis=1)
print(confusion_matrix(y_val2,y_pred))
print('Classification Report')
target_names = ['California','Connecticut','Florida','Masachusetts','Michigan','New York','Others','USA','Virginia','Washington']
print(classification_report(y_val2, y_pred, target_names=target_names))

Confusion matrix - Test
[[145   0   0   2  20   3  16   6   8  16]
 [  0   0   0   1   0  14   2   5   3  14]
 [  3   0   0   0   0  35   4   1   0   0]
 [  8   0   0  59   1   7   7   9   2  29]
 [ 10   0   0   3 118   4   1   4   1   3]
 [  5   0   0   1   0  59   9   3   0   2]
 [  9   0   0   4   1   5  39  42   6  25]
 [  8   0   0   5   5   9  24 135   3  48]
 [ 20   0   0   5   2   2   1   7  54  17]
 [ 18   0   0  21   1   4   8  36   8 465]]
Classification Report
              precision    recall  f1-score   support

  California       0.64      0.67      0.66       216
 Connecticut       0.00      0.00      0.00        39
     Florida       0.00      0.00      0.00        43
Masachusetts       0.58      0.48      0.53       122
    Michigan       0.80      0.82      0.81       144
    New York       0.42      0.75      0.53        79
      Others       0.35      0.30      0.32       131
         USA       0.54      0.57      0.56       237
    Virginia       0.64      0.50   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
